# Appendix 10.2: Tool Use

- [Lesson](#lesson)
- [Exercises](#exercises)
- [Example Playground](#example-playground)

## Setup

Run the following setup cell to load your API key and establish the `get_completion` helper function.

In [ ]:
from dotenv import load_dotenv
import os
import re
from openai import OpenAI

# Load variables from .env
load_dotenv()

# Access variables
API_KEY = os.getenv("API_KEY")
BASE_URL = "https://api.deepseek.com"
MODEL_NAME = "deepseek-chat"

# Store the API_KEY, BASE_URL & MODEL_NAME variables for use across notebooks within the IPython store
%store API_KEY
%store BASE_URL
%store MODEL_NAME

# Initialize the client
client = OpenAI(api_key=API_KEY, base_url=BASE_URL)

# Helper function to get completions from DeepSeek
def get_completion(messages, system_prompt="", prefill="", stop_sequences=None):
    # Add system prompt as a system message if provided
    if system_prompt:
        full_messages = [{"role": "system", "content": system_prompt}] + messages
    else:
        full_messages = messages
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=full_messages,
        max_tokens=2000,
        temperature=0.0,
        stop=stop_sequences
    )
    
    return response.choices[0].message.content

---

## Lesson

While it might seem conceptually complex at first, tool use, a.k.a. function calling, is actually quite simple! You already know all the skills necessary to implement tool use, which is really just a combination of substitution and prompt chaining.

In previous substitution exercises, we substituted text into prompts. With tool use, we substitute tool or function results into prompts. The model can't literally call or access tools and functions. Instead, we have the model:
1. Output the tool name and arguments it wants to call
2. Halt any further response generation while the tool is called
3. Then we reprompt with the appended tool results

Function calling is useful because it expands the model's capabilities and enables it to handle much more complex, multi-step tasks.
Some examples of functions you can give the model:
- Calculator
- Word counter
- SQL database querying and data retrieval
- Weather API

You can get the model to do tool use by combining these two elements:

1. A system prompt, in which we give the model an explanation of the concept of tool use as well as a detailed descriptive list of the tools it has access to
2. The control logic with which to orchestrate and execute the model's tool use requests

### Examples

To enable tool use with DeepSeek, we start with the system prompt. In this special tool use system prompt, we tell the model:
* The basic premise of tool use and what it entails
* How it can call and use the tools it's been given
* A detailed list of tools it has access to in this specific scenario 

Here's the first part of the system prompt, explaining tool use to the model. This part of the system prompt is generalizable across all instances of prompting for tool use.

In [2]:
system_prompt_tools_general_explanation = """You have access to a set of tools you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

To call a function, respond with a JSON object matching the following schema:
{
  "tool_calls": [
    {
      "type": "function",
      "function": {
        "name": "FUNCTION_NAME",
        "arguments": {
          "PARAM_NAME": "PARAM_VALUE",
          ...
        }
      }
    }
  ]
}

The output and/or any errors will appear in a subsequent reply from the user, and you should use that information to continue your response.
If you need to call multiple functions, you should only call one at a time and wait for its response before calling another function."""

Here's the second part of the system prompt, which defines the exact tools the model has access to in this specific situation. In this example, we will be giving it a calculator tool, which takes three parameters: two operands and an operator. 

Then we combine the two parts of the system prompt.

In [3]:
system_prompt_tools_specific_tools = """Here are the functions available:

calculator: Calculator function for doing basic arithmetic.
  Parameters:
    - first_operand (integer): First operand (before the operator)
    - second_operand (integer): Second operand (after the operator)
    - operator (string): The operation to perform. Must be either +, -, *, or /
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

Now we can give the model a question that requires use of the `calculator` tool. We will use a stop sequence to detect if and when it calls the function.

In [4]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

# Get the model's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt)
print(function_calling_response)

{
  "tool_calls": [
    {
      "type": "function",
      "function": {
        "name": "calculator",
        "arguments": {
          "first_operand": 1984135,
          "second_operand": 9343116,
          "operator": "*"
        }
      }
    }
  ]
}


Now, we can extract out the parameters from the model's function call and actually run the function on its behalf.

First we'll define the function's code.

In [5]:
import json

def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

Then we'll extract the parameters from the model's function call response. If all the parameters exist, we run the calculator tool.

In [6]:
def extract_function_call(response_text):
    # Find the JSON object in the response
    try:
        # Try to find JSON within the text using regex
        json_match = re.search(r'\{\s*"tool_calls".*\}', response_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
            return json.loads(json_str)
        else:
            # If no JSON with tool_calls found, try to parse the entire response as JSON
            return json.loads(response_text)
    except json.JSONDecodeError:
        print("Could not parse JSON from response")
        return None

# Extract function call details
function_call = extract_function_call(function_calling_response)
if function_call and 'tool_calls' in function_call:
    function_name = function_call['tool_calls'][0]['function']['name']
    arguments = function_call['tool_calls'][0]['function']['arguments']
    
    if function_name == 'calculator':
        if isinstance(arguments, str):
            arguments = json.loads(arguments)
            
        first_operand = arguments.get('first_operand')
        second_operand = arguments.get('second_operand')
        operator = arguments.get('operator')
        
        if first_operand is not None and second_operand is not None and operator:
            result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
            print("---------------- RESULT ----------------")
            print(f"{result:,}")

---------------- RESULT ----------------
18,538,003,464,660


Now that we have a result, we have to properly format that result so that when we pass it back to the model, it understands what tool that result is in relation to. 

In [7]:
def format_tool_response(function_name, result):
    return f"Function '{function_name}' returned: {result}"

function_results = format_tool_response('calculator', result) if 'result' in locals() else ""
print(function_results)

Function 'calculator' returned: 18538003464660


Now all we have to do is send this result back to the model by appending the result to the same message chain as before, and we're good!

In [8]:
# Construct the full conversation
messages = [
    multiplication_message,
    {"role": "assistant", "content": function_calling_response},
    {"role": "user", "content": function_results}
]
   
# Print the model's response
final_response = get_completion(messages, system_prompt=system_prompt)
print("------------- FINAL RESULT -------------")
print(final_response)

------------- FINAL RESULT -------------
The result of multiplying **1,984,135** by **9,343,116** is **18,538,003,464,660**. Let me know if you need further assistance! 😊


Congratulations on running an entire tool use chain end to end!

Now what if we give the model a question that doesn't require using the given tool at all?

In [9]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

# Get the model's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt)
print(function_calling_response)

The capital of France is Paris. Let me know if you need more information!


Success! As you can see, the model knew not to call the function when it wasn't needed.

If you would like to experiment with the lesson prompts without changing any content above, scroll all the way to the bottom of the lesson notebook to visit the [**Example Playground**](#example-playground).

---

## Exercises
- [Exercise 10.2.1 - SQL](#exercise-1021---SQL)

### Exercise 10.2.1 - SQL
In this exercise, you'll be writing a tool use prompt for querying and writing to the world's smallest "database". Here's the initialized database, which is really just a dictionary.

In [10]:
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

And here is the code for the functions that write to and from the database.

In [11]:
def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

To solve the exercise, start by defining a system prompt like `system_prompt_tools_specific_tools` above. Make sure to include the name and description of each tool, along with the name and type and description of each parameter for each function. We've given you some starting scaffolding below.

In [13]:
system_prompt_tools_specific_tools_sql = """Here are the functions available:
Get user: Returns the name of a user given its user ID
    Name: get_user
    Parameters: 
        - user_id (integer): ID of the desired user name

Get product: Returns the name of a product given its product ID
    Name: get_product
    Paramters:
        - product_id (integer): ID of the desired product

Add user: Appends a new user to our users database with new email and name
    Name: add_user
    Parameters:
        - name (string): Name of the new user
        - email (string): Email of the new user

Add product: Appends a new product to our products database
    Name: add_product
    Paramters:
        - name (string): Name of the new product
        - price (integer): Price of the new product
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools_sql

When you're ready, you can try out your tool definition system prompt on the examples below. Just run the below cell!

In [19]:
db = {
    "users": [
        {"id": 1, "name": "Alice", "email": "alice@example.com"},
        {"id": 2, "name": "Bob", "email": "bob@example.com"},
        {"id": 3, "name": "Charlie", "email": "charlie@example.com"}
    ],
    "products": [
        {"id": 1, "name": "Widget", "price": 9.99},
        {"id": 2, "name": "Gadget", "price": 14.99},
        {"id": 3, "name": "Doohickey", "price": 19.99}
    ]
}

def get_user(user_id):
    for user in db["users"]:
        if user["id"] == user_id:
            return user
    return None

def get_product(product_id):
    for product in db["products"]:
        if product["id"] == product_id:
            return product
    return None

def add_user(name, email):
    user_id = len(db["users"]) + 1
    user = {"id": user_id, "name": name, "email": email}
    db["users"].append(user)
    return user

def add_product(name, price):
    product_id = len(db["products"]) + 1
    product = {"id": product_id, "name": name, "price": price}
    db["products"].append(product)
    return product

def extract_function_call(response_text):
    # Find the JSON object in the response
    try:
        # Try to find JSON within the text using regex
        json_match = re.search(r'\{\s*"tool_calls".*\}', response_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
            return json.loads(json_str)
        else:
            # If no JSON with tool_calls found, try to parse the entire response as JSON
            return json.loads(response_text)
    except json.JSONDecodeError:
        print("Could not parse JSON from response")
        return None

examples = [
    "Add a user to the database named Deborah.",
    "Add a product to the database named Thingo",
    "Tell me the name of User 2",
    "Tell me the name of Product 3"
]

for example in examples:
    message = {
        "role": "user",
        "content": example
    }

    # Get & print the model's response
    function_calling_response = get_completion([message], system_prompt=system_prompt)
    print(example, "\n----------\n\n", function_calling_response, "\n*********\n*********\n*********\n\n")

    function_call = extract_function_call(function_calling_response)
    if function_call and 'tool_calls' in function_call:
        function_name = function_call['tool_calls'][0]['function']['name']
        arguments = function_call['tool_calls'][0]['function']['arguments']
        if function_name == "get_user":
            if isinstance(arguments, str):
                arguments = json.loads(arguments)
            user_id = arguments.get('user_id')
            print(f"The user is: {get_user(user_id).get('name')}")

Add a user to the database named Deborah. 
----------

 {
  "tool_calls": [
    {
      "type": "function",
      "function": {
        "name": "add_user",
        "arguments": {
          "name": "Deborah",
          "email": ""
        }
      }
    }
  ]
} 
*********
*********
*********


Add a product to the database named Thingo 
----------

 {
  "tool_calls": [
    {
      "type": "function",
      "function": {
        "name": "add_product",
        "arguments": {
          "name": "Thingo",
          "price": 0
        }
      }
    }
  ]
} 
*********
*********
*********


Tell me the name of User 2 
----------

 {
  "tool_calls": [
    {
      "type": "function",
      "function": {
        "name": "get_user",
        "arguments": {
          "user_id": 2
        }
      }
    }
  ]
} 
*********
*********
*********


The user is: Bob
Tell me the name of Product 3 
----------

 {
  "tool_calls": [
    {
      "type": "function",
      "function": {
        "name": "get_product"

If you did it right, the function calling messages should call the `add_user`, `add_product`, `get_user`, and `get_product` functions correctly.

For extra credit, add some code cells and write parameter-parsing code. Then call the functions with the parameters the model gives you to see the state of the "database" after the call.

---

## Example Playground

This is an area for you to experiment freely with the prompt examples shown in this lesson and tweak prompts to see how it may affect the model's responses.

In [ ]:
system_prompt_tools_general_explanation = """You have access to a set of tools you can use to answer the user's question. This includes access to a
sandboxed computing environment. You do NOT currently have the ability to inspect files or interact with external
resources, except by invoking the below functions.

To call a function, respond with a JSON object matching the following schema:
{
  "tool_calls": [
    {
      "type": "function",
      "function": {
        "name": "FUNCTION_NAME",
        "arguments": {
          "PARAM_NAME": "PARAM_VALUE",
          ...
        }
      }
    }
  ]
}

The output and/or any errors will appear in a subsequent reply from the user, and you should use that information to continue your response.
If you need to call multiple functions, you should only call one at a time and wait for its response before calling another function."""

In [ ]:
system_prompt_tools_specific_tools = """Here are the functions available:

calculator: Calculator function for doing basic arithmetic.
  Parameters:
    - first_operand (integer): First operand (before the operator)
    - second_operand (integer): Second operand (after the operator)
    - operator (string): The operation to perform. Must be either +, -, *, or /
"""

system_prompt = system_prompt_tools_general_explanation + system_prompt_tools_specific_tools

In [ ]:
multiplication_message = {
    "role": "user",
    "content": "Multiply 1,984,135 by 9,343,116"
}

# Get the model's response
function_calling_response = get_completion([multiplication_message], system_prompt=system_prompt)
print(function_calling_response)

In [ ]:
def do_pairwise_arithmetic(num1, num2, operation):
    if operation == '+':
        return num1 + num2
    elif operation == "-":
        return num1 - num2
    elif operation == "*":
        return num1 * num2
    elif operation == "/":
        return num1 / num2
    else:
        return "Error: Operation not supported."

In [ ]:
def extract_function_call(response_text):
    # Find the JSON object in the response
    try:
        # Try to find JSON within the text using regex
        json_match = re.search(r'\{\s*"tool_calls".*\}', response_text, re.DOTALL)
        if json_match:
            json_str = json_match.group(0)
            return json.loads(json_str)
        else:
            # If no JSON with tool_calls found, try to parse the entire response as JSON
            return json.loads(response_text)
    except json.JSONDecodeError:
        print("Could not parse JSON from response")
        return None

# Extract function call details
function_call = extract_function_call(function_calling_response)
if function_call and 'tool_calls' in function_call:
    function_name = function_call['tool_calls'][0]['function']['name']
    arguments = function_call['tool_calls'][0]['function']['arguments']
    
    if function_name == 'calculator':
        if isinstance(arguments, str):
            arguments = json.loads(arguments)
            
        first_operand = arguments.get('first_operand')
        second_operand = arguments.get('second_operand')
        operator = arguments.get('operator')
        
        if first_operand is not None and second_operand is not None and operator:
            result = do_pairwise_arithmetic(int(first_operand), int(second_operand), operator)
            print("---------------- RESULT ----------------")
            print(f"{result:,}")

In [ ]:
def format_tool_response(function_name, result):
    return f"Function '{function_name}' returned: {result}"

function_results = format_tool_response('calculator', result) if 'result' in locals() else ""
print(function_results)

In [ ]:
# Construct the full conversation
messages = [
    multiplication_message,
    {"role": "assistant", "content": function_calling_response},
    {"role": "user", "content": function_results}
]
   
# Print the model's response
final_response = get_completion(messages, system_prompt=system_prompt)
print("------------- FINAL RESULT -------------")
print(final_response)

In [ ]:
non_multiplication_message = {
    "role": "user",
    "content": "Tell me the capital of France."
}

# Get the model's response
function_calling_response = get_completion([non_multiplication_message], system_prompt=system_prompt)
print(function_calling_response)